The data can be directly downloaded from the [ScPCA Portal, from Alex's Lemonade Stand Foundation](https://scpca.alexslemonade.org/projects/SCPCP000005)

In [1]:
# import the dependencies
import sys

# general imports
import single_translator_VAE as sv
from importlib import reload

import os
import pandas as pd
import scanpy as sc

sys.path.insert(1, "../../")
sys.path.insert(1, "../")
sys.path.insert(1, "../../../../../")
reload(sv)

Now reading in the other set of Anndatas from the Single-Cell Atlas:

In [2]:
res_name = "RMS"
# Directory containing all the folders
path = f"{os.getcwd()}/../../data/{res_name}/"
base_dir = f"{os.getcwd()}/../../data/{res_name}/SCPCP000005_SINGLE_CELL_ANN_DATA_2024-08-01"

In [3]:
# List to store the AnnData objects
anndata_list = []

# Loop through each folder in the base directory
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.isdir(folder_path):
        # Look for the _processed_rna.h5ad file in the folder
        for file_name in os.listdir(folder_path):
            if file_name.endswith("_processed_rna.h5ad"):
                file_path = os.path.join(folder_path, file_name)
                print(f"Reading {file_path}...")
                adata = sc.read_h5ad(file_path)
                anndata_list.append(adata)

# Check the number of AnnData objects read
print(f"Total AnnData objects read: {len(anndata_list)}")

Reading /Users/ivicha/Documents/snsc_VAE/single_translator_VAE/docs/notebooks/../../data/RMS/SCPCP000005_SINGLE_CELL_ANN_DATA_2024-08-01/SCPCS000277/SCPCL000503_processed_rna.h5ad...
Reading /Users/ivicha/Documents/snsc_VAE/single_translator_VAE/docs/notebooks/../../data/RMS/SCPCP000005_SINGLE_CELL_ANN_DATA_2024-08-01/SCPCS000283/SCPCL000509_processed_rna.h5ad...
Reading /Users/ivicha/Documents/snsc_VAE/single_translator_VAE/docs/notebooks/../../data/RMS/SCPCP000005_SINGLE_CELL_ANN_DATA_2024-08-01/SCPCS000284/SCPCL000510_processed_rna.h5ad...
Reading /Users/ivicha/Documents/snsc_VAE/single_translator_VAE/docs/notebooks/../../data/RMS/SCPCP000005_SINGLE_CELL_ANN_DATA_2024-08-01/SCPCS000270/SCPCL000496_processed_rna.h5ad...
Reading /Users/ivicha/Documents/snsc_VAE/single_translator_VAE/docs/notebooks/../../data/RMS/SCPCP000005_SINGLE_CELL_ANN_DATA_2024-08-01/SCPCS000279/SCPCL000505_processed_rna.h5ad...
Reading /Users/ivicha/Documents/snsc_VAE/single_translator_VAE/docs/notebooks/../../d

In [4]:
# Find the intersection of all gene names (variables) across all AnnData objects
common_genes = set(anndata_list[0].var_names)
for adata in anndata_list[1:]:
    common_genes &= set(adata.var_names)

# Convert to a list and sort
common_genes = list(common_genes)
common_genes.sort()
print(f"Number of common genes: {len(common_genes)}")

# Filter each AnnData object to only include the common genes
filtered_anndata_list = []
for adata in anndata_list:
    # Subset the AnnData object to only the common genes
    adata_filtered = adata[:, common_genes].copy()
    filtered_anndata_list.append(adata_filtered)

# Check the dimensions of the filtered datasets
for i, adata_filtered in enumerate(filtered_anndata_list):
    print(f"Dataset {i}: {adata_filtered.shape}")

Number of common genes: 60319
Dataset 0: (9233, 60319)
Dataset 1: (7656, 60319)
Dataset 2: (8858, 60319)
Dataset 3: (5439, 60319)
Dataset 4: (4539, 60319)
Dataset 5: (4407, 60319)
Dataset 6: (369, 60319)
Dataset 7: (3851, 60319)
Dataset 8: (7385, 60319)
Dataset 9: (8173, 60319)
Dataset 10: (8135, 60319)
Dataset 11: (6665, 60319)
Dataset 12: (9085, 60319)
Dataset 13: (9109, 60319)
Dataset 14: (5213, 60319)
Dataset 15: (3519, 60319)
Dataset 16: (4797, 60319)
Dataset 17: (2485, 60319)
Dataset 18: (4198, 60319)
Dataset 19: (5396, 60319)
Dataset 20: (11906, 60319)
Dataset 21: (5954, 60319)
Dataset 22: (4820, 60319)
Dataset 23: (6133, 60319)
Dataset 24: (7178, 60319)
Dataset 25: (4224, 60319)
Dataset 26: (5382, 60319)
Dataset 27: (5748, 60319)
Dataset 28: (5098, 60319)
Dataset 29: (1129, 60319)
Dataset 30: (4305, 60319)
Dataset 31: (5615, 60319)
Dataset 32: (6371, 60319)
Dataset 33: (9139, 60319)
Dataset 34: (753, 60319)
Dataset 35: (4059, 60319)
Dataset 36: (105, 60319)
Dataset 37: (4817, 6

In [5]:
# Concatenate all the filtered AnnData objects into a single AnnData object
combined_adata = sc.concat(filtered_anndata_list, axis=0)

/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
# We don't want log counts for now:
combined_adata.layers["counts"] = combined_adata.layers["spliced"]
combined_adata.X = combined_adata.layers["spliced"]

# Check the dimensions of the combined AnnData object
print(f"Combined AnnData shape: {combined_adata.shape}")

/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Combined AnnData shape: (313712, 60319)


We'll remove the version of 10Xv3.1, keeping only 10Xv2 and 10Xv3.

In [6]:
# Sample IDs to be removed
sample_ids_to_remove = [
    "SCPCS000704",
    "SCPCS000707",
    "SCPCS000709",
    "SCPCS000710",
    "SCPCS000711",
    "SCPCS000712",
    "SCPCS000713",
    "SCPCS000714",
    "SCPCS000715",
    "SCPCS000716",
    "SCPCS000717",
]
# Remove the specified sample_ids from combined_adata based on scpca_sample_ids
combined_adata = combined_adata[~combined_adata.obs.sample_id.isin(sample_ids_to_remove)]

In [7]:
combined_adata.obs["batch"] = combined_adata.obs["sample_id"]
combined_adata.obs["patient_id"] = combined_adata.obs["participant_id"]
combined_adata.obs["data_type"] = combined_adata.obs["suspension_type"].cat.rename_categories(
    {"cell": "single_cell", "nucleus": "single_nucleus"}
)
combined_adata.obs["cell_types"] = combined_adata.obs.singler_celltype_annotation
print(combined_adata.obs.cell_types.value_counts())
print(combined_adata.obs.cell_types.unique())

/var/folders/d3/x10lxf7x1n37__s3n36qxmlw0000gp/T/ipykernel_37417/2257091691.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  combined_adata.obs["batch"] = combined_adata.obs["sample_id"]


cell_types
muscle cell                                        66633
cell of skeletal muscle                            51640
fibroblast                                         23252
astrocyte                                          18984
neuron                                             18731
chondrocyte                                        15878
smooth muscle cell                                 12844
mesangial cell                                     12224
macrophage                                          3916
pericyte                                            3882
endothelial cell                                    3450
erythrocyte                                         2919
fat cell                                            2138
microvascular endothelial cell                       953
epithelial cell                                      951
monocyte                                             940
hematopoietic stem cell                              557
common lymphoid prog

/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


We'll remove the version of 10Xv3.1, keeping only 10Xv2 and 10Xv3.

Create a mapping dictionary to get more "broad" cell types

In [8]:
cell_type_mapping = {
    "neuron": "neuron",
    "smooth muscle cell": "muscle cell",
    "macrophage": "macrophage",
    "endothelial cell": "endothelial cell",
    "chondrocyte": "chondrocyte",  # No direct match
    "fat cell": "fat cell",  # No direct match
    "melanocyte": "melanocyte",
    "monocyte": "monocyte",  # Map to a broader category
    "fibroblast": "fibroblast",
    "muscle cell": "muscle cell",  # Closest category
    "cell of skeletal muscle": "muscle cell",  # Closest category
    "hematopoietic stem cell": "hematopoietic progenitor cell",  # No direct match
    "microvascular endothelial cell": "endothelial cell",
    "pericyte": "pericyte",  # Closest category
    "astrocyte": "astrocyte",  # Closest category
    "CD4-positive, alpha-beta T cell": "T cell",
    "erythrocyte": "erythrocyte",  # No direct match
    "common lymphoid progenitor": "common lymphoid progenitor",  # No direct match
    "mesangial cell": "mesangial cell",  # No direct match
    "dendritic cell": "Macrophage",  # Closest category
    "natural killer cell": "T cell",  # Include in T cells
    "epithelial cell": "epithelial cell",  # No direct match
    "plasma cell": "plasma cell",  # No direct match
    "alternatively activated macrophage": "macrophage",
    "effector memory CD8-positive, alpha-beta T cell": "T cell",
    "megakaryocyte": "megakaryocyte",  # No direct match
    "central memory CD4-positive, alpha-beta T cell": "T cell",
    "regulatory T cell": "T cell",
    "naive B cell": "B cell",  # No direct match
    "hematopoietic multipotent progenitor cell": "hematopoietic progenitor cell",  # No direct match
    "granulocyte monocyte progenitor cell": "granulocyte monocyte progenitor cell",  # No direct match
    "neutrophil": "neutrophil",  # No direct match
    "CD8-positive, alpha-beta T cell": "T cell",
    "central memory CD8-positive, alpha-beta T cell": "T cell",
    "effector memory CD4-positive, alpha-beta T cell": "T cell",
    "megakaryocyte-erythroid progenitor cell": "megakaryocyte-erythroid progenitor cell",  # No direct match
    "inflammatory macrophage": "macrophage",
    "eosinophil": "eosinophil",  # No direct match
    "class switched memory B cell": "B cell",  # No direct match
    "Unclassified cell": None,  # No direct match
    "common myeloid progenitor": "common myeloid progenitor",  # No direct match
    "memory B cell": "B cell",  # No direct match
}

# Map new cell types to previous experiment's categories
combined_adata.obs["cell_types"] = combined_adata.obs["singler_celltype_annotation"].map(cell_type_mapping)

# Drop None (no direct match)
combined_adata = combined_adata[~combined_adata.obs["cell_types"].isnull()].copy()

# Check the first few rows of the filtered data
print(combined_adata.obs[["singler_celltype_annotation", "cell_types"]].head())

                 singler_celltype_annotation   cell_types
ACAAAGATCCTTCGAC                 chondrocyte  chondrocyte
CATTCCGTCATCCTGC     cell of skeletal muscle  muscle cell
ACCTACCCAGCCTATA                 chondrocyte  chondrocyte
GCCATTCAGGCTGGAT                  macrophage   macrophage
TAATTCCTCCATTCGC                  macrophage   macrophage


/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 242293 × 60319
    obs: 'barcodes', 'sum', 'detected', 'subsets_mito_sum', 'subsets_mito_detected', 'subsets_mito_percent', 'total', 'prob_compromised', 'scpca_filter', 'sizeFactor', 'cluster', 'singler_celltype_ontology', 'singler_celltype_annotation', 'cellassign_celltype_annotation', 'cellassign_max_prediction', 'library_id', 'sample_id', 'scpca_project_id', 'submitter_id', 'participant_id', 'submitter', 'age', 'sex', 'diagnosis', 'subdiagnosis', 'tissue_location', 'disease_timing', 'organism', 'is_xenograft', 'is_cell_line', 'development_stage_ontology_term_id', 'sex_ontology_term_id', 'organism_ontology_id', 'self_reported_ethnicity_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id', 'assay_ontology_term_id', 'suspension_type', 'is_primary_data', 'batch', 'patient_id', 'data_type', 'cell_types'
    obsm: 'X_PCA', 'X_UMAP'
    layers: 'spliced', 'counts'

Removing cell types with less than 200 cells (there are over 30 datasets, less than 200 cells is very rare)

In [9]:
# Count the number of cells in each cell type
cell_counts = combined_adata.obs["cell_types"].value_counts()

# Identify cell types with at least 100 cells
valid_cell_types = cell_counts[cell_counts >= 1000].index

# Filter the AnnData object to keep only the cells from these cell types
combined_adata = combined_adata[combined_adata.obs["cell_types"].isin(valid_cell_types)].copy()

print(combined_adata.obs.cell_types.value_counts())

/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


cell_types
muscle cell         131117
fibroblast           23252
astrocyte            18984
neuron               18731
chondrocyte          15878
mesangial cell       12224
endothelial cell      4403
macrophage            3980
pericyte              3882
erythrocyte           2919
fat cell              2138
Name: count, dtype: int64

## Finally, creating our training data:

In [10]:
# We'll save to convert in https://www.biotools.fr/human/ensembl_symbol_converter
gene_names = combined_adata.var_names.values
# Define the file path
file_path = f"{path}/gene_names.csv"
# Convert the list to a DataFrame and save as CSV
gene_df = pd.DataFrame(gene_names, columns=["Gene"])
gene_df.to_csv(file_path, index=False)

In [11]:
# importing mapping
file_path = f"{path}gene_names_matched"
gene_df = pd.DataFrame(gene_names, columns=["Gene"])
genes_matched = pd.read_csv(file_path, sep="\t", header=None)
genes_matched.columns = ["Ensembl_ID", "Gene_Name"]

In [12]:
# Create a dictionary for mapping Ensembl IDs to new gene names
gene_name_mapping = pd.Series(genes_matched["Gene_Name"].values, index=genes_matched["Ensembl_ID"]).to_dict()

# Map the Ensembl IDs to new gene names in the AnnData object
new_var_names = combined_adata.var.index.to_series().map(gene_name_mapping)

# Drop genes that do not have a mapping (i.e., NaN values in new_var_names)
valid_var_names = new_var_names.dropna()
combined_adata = combined_adata[:, valid_var_names.index]

# Update the var index (gene names) with the new names
combined_adata.var.index = valid_var_names

# Check the updated var names
print(combined_adata.var.index)
print(combined_adata.var_names_make_unique())

Index(['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH', 'FUCA2',
       'GCLC', 'NFYA',
       ...
       'AC021097.2', 'AL590381.1', 'AC003043.2', 'AC135068.11', 'AL356417.3',
       'AC104389.6', 'AC010616.2', 'AL034430.1', 'AP000646.1', 'AP006216.3'],
      dtype='object', length=57607)


/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [13]:
# sc.pp.neighbors(combined_adata)
# sc.tl.umap(combined_adata)
# # Plot UMAP colored by cell type
# sc.pl.umap(combined_adata, color='cell_types', legend_loc='right margin', title='Cell Type Annotations')
# sc.pl.umap(combined_adata, color='data_type', legend_loc='right margin', title='Cell Type Annotations')

In [14]:
# As it is popular to normalize the data for many methods,
# we can use Scanpy for this; however,
# it’s important to keep the count information intact for scvi-tools models.
combined_adata.var_names_make_unique()
combined_adata.obs_names_make_unique()
combined_adata.layers["counts"] = combined_adata.X.copy()

In [15]:
# Count the number of cells in each cell type
cell_counts = combined_adata.obs["cell_types"].value_counts()
# Identify cell types with at least 100 cells
valid_cell_types = cell_counts[cell_counts >= 1000].index
# Filter the AnnData object to keep only the cells from these cell types
combined_adata = combined_adata[combined_adata.obs["cell_types"].isin(valid_cell_types)].copy()
print(combined_adata.obs.cell_types.value_counts())

/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


cell_types
muscle cell         131117
fibroblast           23252
astrocyte            18984
neuron               18731
chondrocyte          15878
mesangial cell       12224
endothelial cell      4403
macrophage            3980
pericyte              3882
erythrocyte           2919
fat cell              2138
Name: count, dtype: int64

Withholding datasets for deconvolution experiment:

In [13]:
# Retrieve the last 4 unique patient_ids for single_nucleus data_type
last_four_patient_ids = combined_adata[combined_adata.obs.data_type == "single_nucleus"].obs.patient_id.unique()[-4:]

In [14]:
# Concatenate data for the last 4 patient_ids
sn_withheld = sc.concat([combined_adata[combined_adata.obs.patient_id == pid] for pid in last_four_patient_ids])

/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [15]:
# Update the 'deconv' column
sn_withheld.obs["deconv"] = "sn_ref"

# Remove these patient_ids from combined_adata
combined_adata = combined_adata[~combined_adata.obs.patient_id.isin(last_four_patient_ids)]

In [16]:
# Retrieve the last 4 unique patient_ids for single_nucleus data_type
last_four_patient_ids = combined_adata[combined_adata.obs.data_type == "single_nucleus"].obs.patient_id.unique()[-4:]

# Concatenate data for the last 4 patient_ids
sc_withheld_1 = sc.concat([combined_adata[combined_adata.obs.patient_id == pid] for pid in last_four_patient_ids])

# Update the 'deconv' column
sc_withheld_1.obs["deconv"] = "sc_ref"

# Remove these patient_ids from combined_adata
combined_adata = combined_adata[~combined_adata.obs.patient_id.isin(last_four_patient_ids)]

/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [17]:
# Retrieve the first 4 unique patient_ids for single_nucleus data_type
first_four_patient_ids = combined_adata[combined_adata.obs.data_type == "single_nucleus"].obs.patient_id.unique()[:4]

# Concatenate data for the last 4 patient_ids
sc_withheld_2 = sc.concat([combined_adata[combined_adata.obs.patient_id == pid] for pid in first_four_patient_ids])

# Update the 'deconv' column
sc_withheld_2.obs["deconv"] = "pseudobulks"

# Remove these patient_ids from combined_adata
combined_adata = combined_adata[~combined_adata.obs.patient_id.isin(first_four_patient_ids)]

/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


And diving into Training and Testing with Remaining Cells:

In [18]:
# and writing into one Anndata:
adata_withheld = sc.concat([sn_withheld, sc_withheld_1, sc_withheld_2])
adata_path = os.path.join(path, f"sc_sn_{res_name}_withheld.h5ad")
adata_withheld.write_h5ad(adata_path)

/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3

In [19]:
combined_adata.obs["TrainTest"] = "Train"
adata_withheld.obs["TrainTest"] = "Test"
adata = sc.concat([combined_adata, adata_withheld])

/var/folders/d3/x10lxf7x1n37__s3n36qxmlw0000gp/T/ipykernel_40125/2183832773.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  combined_adata.obs["TrainTest"] = "Train"
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/Users/ivicha/Documents/snsc_VAE/env_vae/lib/python3.11/site-p

And Saving:

In [20]:
# writing our data:
adata_path = os.path.join(path, f"sc_sn_{res_name}_train.h5ad")
combined_adata.write_h5ad(adata_path)

adata_path = os.path.join(path, f"sc_sn_{res_name}_test.h5ad")
adata_withheld.write_h5ad(adata_path)

: 